# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [1]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import pandas as pd

## html capture

In [2]:
url='https://www.basketball-reference.com/leagues/NBA_2024_standings.html'
response = requests.get(url)
response.status_code

200

## parseamos el html

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')
west_table = soup.find('table', {'id': "confs_standings_W"})
west_table

<table class="suppress_all sortable stats_table" data-cols-to-freeze=",1" id="confs_standings_W">
<caption>  Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="Western Conference" class="poptip sort_default_asc left" data-stat="team_name" scope="col">Western Conference</th>
<th aria-label="Wins" class="poptip right" data-stat="wins" data-tip="Wins" scope="col">W</th>
<th aria-label="Losses" class="poptip right" data-stat="losses" data-tip="Losses" scope="col">L</th>
<th aria-label="Win-Loss Percentage" class="poptip right" data-stat="win_loss_pct" data-tip="Win-Loss Percentage" scope="col">W/L%</th>
<th aria-label="GB" class="poptip sort_default_asc right" data-stat="gb" data-tip="Games Behind" scope="col">GB</th>
<th aria-label="Points Per Game" class="poptip right" data-stat="pts_per_g" data-tip="Points Per Game" scope="col">PS/G</th>
<th aria-label="Opponent Points Per Game" class="poptip right" data-stat="opp_pts_per_g

## dataframe

In [4]:
df = pd.read_html(str(west_table))[0]
df

/tmp/ipykernel_11712/2825806103.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(west_table))[0]


,Western Conference,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Oklahoma City Thunder*,57,25,0.695,—,120.1,112.7,7.36
1,Denver Nuggets*,57,25,0.695,—,114.9,109.6,5.23
2,Minnesota Timberwolves*,56,26,0.683,1.0,113.0,106.5,6.39
3,Los Angeles Clippers*,51,31,0.622,6.0,115.6,112.3,3.41
4,Dallas Mavericks*,50,32,0.610,7.0,117.9,115.6,2.30
5,Phoenix Suns*,49,33,0.598,8.0,116.2,113.2,3.08
6,New Orleans Pelicans*,49,33,0.598,8.0,115.1,110.7,4.46
7,Los Angeles Lakers*,47,35,0.573,10.0,118.0,117.4,1.07
8,Sacramento Kings,46,36,0.561,11.0,116.6,114.8,2.29
9,Golden State Warriors,46,36,0.561,11.0,117.8,115.2,2.77


 ## base de datos

 Creamos la base de datos y guardamos localmente los datos

In [5]:
conn = sqlite3.connect("nba_standings.db") 
cursor = conn.cursor()

df.to_sql("west_standings", conn, if_exists="replace", index=False)

df_sql = pd.read_sql("SELECT * FROM west_standings", conn)

df_sql.head()

,Western Conference,W,L,W/L%,GB,PS/G,PA/G,SRS
0,Oklahoma City Thunder*,57,25,0.695,—,120.1,112.7,7.36
1,Denver Nuggets*,57,25,0.695,—,114.9,109.6,5.23
2,Minnesota Timberwolves*,56,26,0.683,1.0,113.0,106.5,6.39
3,Los Angeles Clippers*,51,31,0.622,6.0,115.6,112.3,3.41
4,Dallas Mavericks*,50,32,0.610,7.0,117.9,115.6,2.30


## Generación de datos informe

Se procesan los datos por valores mayores a 45 pts ganados

In [6]:
query = "SELECT `Western Conference`, W, L FROM west_standings WHERE W > 45"
top_teams = pd.read_sql(query, conn)
top_teams

,Western Conference,W,L
0,Oklahoma City Thunder*,57,25
1,Denver Nuggets*,57,25
2,Minnesota Timberwolves*,56,26
3,Los Angeles Clippers*,51,31
4,Dallas Mavericks*,50,32
5,Phoenix Suns*,49,33
6,New Orleans Pelicans*,49,33
7,Los Angeles Lakers*,47,35
8,Sacramento Kings,46,36
9,Golden State Warriors,46,36


## Cerramos conexión de la db

In [7]:
conn.close()